In [1]:
# evaluate model performance with outliers removed using isolation forest

from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
from keras import models,backend,optimizers,layers
from keras.models import Model
from keras.layers import LSTM,Dense,Dropout,concatenate,Input
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
import sys
import joblib
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt
from sklearn import preprocessing
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time
from keras.models import load_model

def mae_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)

    return sum(abs(y_v[:,d]-p_v[:,d]))/len(y_v)

def mse_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)

    return sum((y_v[:,d]-p_v[:,d])*(y_v[:,d]-p_v[:,d]))/len(y_v)

def mape_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)
    return (1/len(y_v))*(sum(abs((p_v[:,d]-y_v[:,d])/y_v[:,d])))



data_loc= pd.read_csv('history_dec.csv', encoding='utf-8')


#data_loc=data_loc.dropna()
data_loc

,deviceId,location,past_24_pm2.5,past_24_humi,past_24_temp,past_23_pm2.5,past_23_humi,past_23_temp,past_22_pm2.5,past_22_humi,...,15hrs,16hrs,17hrs,18hrs,19hrs,20hrs,21hrs,22hrs,23hrs,24hrs
0,1858224692,0,7.0,0.00,0.00,7.0,0.00,0.00,7.0,0.00,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
1,1858224692,0,7.0,0.00,0.00,7.0,0.00,0.00,7.0,0.00,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
2,1858224692,0,7.0,0.00,0.00,7.0,0.00,0.00,7.0,0.00,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
3,1858224692,0,7.0,0.00,0.00,7.0,0.00,0.00,7.0,0.00,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
4,1858224692,0,7.0,0.00,0.00,7.0,0.00,0.00,7.0,0.00,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902139,7993968814,0,7.0,69.53,20.11,8.0,69.65,19.98,8.0,70.64,...,24.0,12.0,10.0,15.0,10.0,8.0,13.0,9.0,8.0,6.0
902140,7993968814,0,8.0,69.65,19.98,8.0,70.64,20.05,8.0,71.68,...,12.0,10.0,15.0,10.0,8.0,13.0,9.0,8.0,6.0,10.0
902141,7993968814,0,8.0,70.64,20.05,8.0,71.68,19.74,10.0,72.67,...,10.0,15.0,10.0,8.0,13.0,9.0,8.0,6.0,10.0,12.0
902142,7993968814,0,8.0,71.68,19.74,10.0,72.67,19.56,12.0,74.08,...,15.0,10.0,8.0,13.0,9.0,8.0,6.0,10.0,12.0,9.0


In [23]:
##validation_sample=80000 #設定測試集筆數
data_loc=data_loc[data_loc['location']==3]
#data_loc = shuffle(data_loc) #資料亂序
data_loc=data_loc.drop(['deviceId','location'],axis=1)
data_loc

KeyError: 'location'

In [3]:
data_loc=data_loc.drop(data_loc.iloc[:,:48],axis=1)

In [4]:
data_loc=data_loc.drop(data_loc.iloc[:,32:48],axis=1)

In [5]:
data_loc=data_loc.reset_index()

In [2]:
#data_loc['past_1_pm2.5'].isnull()
#data_loc=data_loc.drop(columns=['deviceId','time','location'])
data_loc=data_loc.drop(columns=['index'])

KeyError: "['index'] not found in axis"

In [24]:
for i in range(32):
    mean=data_loc.iloc[:,i].mean()
    data_loc.iloc[:,i]=data_loc.iloc[:,i].fillna(mean)
data_loc=round(data_loc,2)
data_loc.isnull().values.any()

False

In [25]:
data_loc.std()

past_8_pm2.5     7.188246
past_8_humi     13.588339
past_8_temp      4.074520
past_7_pm2.5     7.191354
past_7_humi     13.610854
past_7_temp      4.071287
past_6_pm2.5     7.189177
past_6_humi     13.670674
past_6_temp      4.068811
past_5_pm2.5     7.191247
past_5_humi     13.695182
past_5_temp      4.067404
past_4_pm2.5     7.189619
past_4_humi     13.722146
past_4_temp      4.066635
past_3_pm2.5     7.191740
past_3_humi     13.750066
past_3_temp      4.065474
past_2_pm2.5     7.193830
past_2_humi     13.781955
past_2_temp      4.063760
past_1_pm2.5     7.199624
past_1_humi     13.813105
past_1_temp      4.062261
1hrs             7.206928
2hrs             7.213376
3hrs             7.223352
4hrs             7.234435
5hrs             7.250931
6hrs             7.271850
7hrs             7.291655
8hrs             7.311185
dtype: float64

In [26]:
data_loc.mean()

past_8_pm2.5    14.184768
past_8_humi     78.439514
past_8_temp     19.159245
past_7_pm2.5    14.183886
past_7_humi     78.487756
past_7_temp     19.154330
past_6_pm2.5    14.175329
past_6_humi     78.545409
past_6_temp     19.151460
past_5_pm2.5    14.173834
past_5_humi     78.589520
past_5_temp     19.148258
past_4_pm2.5    14.170492
past_4_humi     78.635936
past_4_temp     19.144625
past_3_pm2.5    14.174262
past_3_humi     78.688683
past_3_temp     19.140641
past_2_pm2.5    14.182023
past_2_humi     78.745032
past_2_temp     19.136264
past_1_pm2.5    14.196712
past_1_humi     78.804360
past_1_temp     19.133578
1hrs            14.210401
2hrs            14.223370
3hrs            14.236618
4hrs            14.250325
5hrs            14.264883
6hrs            14.274870
7hrs            14.274995
8hrs            14.272743
dtype: float64

In [27]:
data_loc.max()

past_8_pm2.5     36.0
past_8_humi     120.0
past_8_temp      35.0
past_7_pm2.5     36.0
past_7_humi     120.0
past_7_temp      35.0
past_6_pm2.5     36.0
past_6_humi     121.0
past_6_temp      35.0
past_5_pm2.5     36.0
past_5_humi     121.0
past_5_temp      35.0
past_4_pm2.5     36.0
past_4_humi     121.0
past_4_temp      35.0
past_3_pm2.5     36.0
past_3_humi     121.0
past_3_temp      35.0
past_2_pm2.5     36.0
past_2_humi     121.0
past_2_temp      35.0
past_1_pm2.5     36.0
past_1_humi     121.0
past_1_temp      35.0
1hrs             36.0
2hrs             36.0
3hrs             36.0
4hrs             36.0
5hrs             36.0
6hrs             36.0
7hrs             36.0
8hrs             36.0
dtype: float64

In [28]:
for i in range(32):
    mean=data_loc.iloc[:,i].mean()
    std=data_loc.iloc[:,i].std()
    new=mean+3*std
    fliter = (data_loc.iloc[:,i]>new)
    fliter1=(data_loc.iloc[:,i]<=3)
    data_loc.iloc[:,i][fliter]=new
    data_loc.iloc[:,i][fliter1]=mean


In [29]:
data_loc.std()

past_8_pm2.5     7.168698
past_8_humi     13.556042
past_8_temp      3.887476
past_7_pm2.5     7.172462
past_7_humi     13.583283
past_7_temp      3.883107
past_6_pm2.5     7.169123
past_6_humi     13.611485
past_6_temp      3.879762
past_5_pm2.5     7.171559
past_5_humi     13.640862
past_5_temp      3.877968
past_4_pm2.5     7.169280
past_4_humi     13.673087
past_4_temp      3.876622
past_3_pm2.5     7.172158
past_3_humi     13.706630
past_3_temp      3.874855
past_2_pm2.5     7.175315
past_2_humi     13.744785
past_2_temp      3.872340
past_1_pm2.5     7.183602
past_1_humi     13.782219
past_1_temp      3.870111
1hrs             7.193723
2hrs             7.202700
3hrs             7.216038
4hrs             7.230795
5hrs             7.250931
6hrs             7.271850
7hrs             7.291655
8hrs             7.311185
dtype: float64

In [19]:
data_loc=round(data_loc,0)
data_loc

,past_8_pm2.5,past_8_humi,past_8_temp,past_7_pm2.5,past_7_humi,past_7_temp,past_6_pm2.5,past_6_humi,past_6_temp,past_5_pm2.5,...,past_1_humi,past_1_temp,1hrs,2hrs,3hrs,4hrs,5hrs,6hrs,7hrs,8hrs
0,7.0,83.0,27.0,7.0,83.0,27.0,7.0,83.0,27.0,7.0,...,83.0,27.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
1,7.0,83.0,27.0,7.0,83.0,27.0,7.0,83.0,27.0,7.0,...,83.0,27.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
2,7.0,83.0,27.0,7.0,83.0,27.0,7.0,83.0,27.0,7.0,...,83.0,27.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
3,7.0,83.0,27.0,7.0,83.0,27.0,7.0,83.0,27.0,7.0,...,83.0,27.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
4,7.0,83.0,27.0,7.0,83.0,27.0,7.0,83.0,27.0,7.0,...,83.0,27.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319823,24.0,89.0,19.0,17.0,90.0,19.0,22.0,90.0,19.0,15.0,...,91.0,19.0,19.0,17.0,14.0,14.0,16.0,16.0,22.0,15.0
319824,17.0,90.0,19.0,22.0,90.0,19.0,15.0,90.0,19.0,19.0,...,91.0,19.0,17.0,14.0,14.0,16.0,16.0,22.0,15.0,16.0
319825,22.0,90.0,19.0,15.0,90.0,19.0,19.0,91.0,19.0,12.0,...,91.0,19.0,14.0,14.0,16.0,16.0,22.0,15.0,16.0,9.0
319826,15.0,90.0,19.0,19.0,91.0,19.0,12.0,91.0,19.0,14.0,...,92.0,19.0,14.0,16.0,16.0,22.0,15.0,16.0,9.0,7.0


In [30]:
data_loc.min()

past_8_pm2.5    4.0
past_8_humi     4.0
past_8_temp     4.0
past_7_pm2.5    4.0
past_7_humi     4.0
past_7_temp     4.0
past_6_pm2.5    4.0
past_6_humi     4.0
past_6_temp     4.0
past_5_pm2.5    4.0
past_5_humi     4.0
past_5_temp     4.0
past_4_pm2.5    4.0
past_4_humi     4.0
past_4_temp     4.0
past_3_pm2.5    4.0
past_3_humi     4.0
past_3_temp     4.0
past_2_pm2.5    4.0
past_2_humi     4.0
past_2_temp     4.0
past_1_pm2.5    4.0
past_1_humi     4.0
past_1_temp     4.0
1hrs            4.0
2hrs            4.0
3hrs            4.0
4hrs            4.0
5hrs            4.0
6hrs            4.0
7hrs            4.0
8hrs            4.0
dtype: float64

In [21]:
data_loc.to_csv("D:/download/完成/final0.csv",index=False)